Until [afni/afni#156 (🐍 lib_afni1D — Python 2→3)](https://github.com/afni/afni/pull/156) is resolved,

```bash
pip install "git+https://github.com/shnizzedy/afni.git@lib_afni1D/2→3#egg=afni-python&subdirectory=src/python_scripts"
```

to pip-install a Python 3 compatible `afni_python`.

In [ ]:
%load_ext autoreload
%autoreload 2

# 🧙 magics

In [ ]:
# imports
import git
import numpy as np
import os
import pandas as pd
import yaml

from afni_python.lib_afni1D import Afni1D
from string import ascii_lowercase

from compare import get_paths
from defaults import motion_list, regressor_list
from subjects import fmriprep_sub, generate_subject_list_for_range

In [ ]:
# function definitions

def read_feature(file, feature, software="C-PAC"):
    """
    
    """
    software = "C-PAC" if software.lower() in ["c-pac", "cpac"] else software.lower()
    
    feature_label = feature_headers.get(feature, {}).get(software, '') if (
        "CompCor" not in feature
    ) else f"{feature[:-1]}PC{feature[-1]}" if (
        software=="C-PAC"
    ) else f"{feature[0]}_comp_cor_0{feature[-1]}" if (
        software=="fmriprep"
    ) else ""
    
    if software=="C-PAC":
        data = Afni1D(file)
        header = data.header[-1] if len(data.header) else ""
        header_list = header.split('\t')
        return(
            data.mat[header_list.index(feature_label)] if (
                feature_label in header_list
            ) else data.mat[0][1:]
        )
    
    elif software=="fmriprep":
        if file.endswith('.tsv'):
            data = pd.read_csv(file, sep='\t')
            if feature_label in data.columns:
                return(data[feature_label])
        elif file.endswith('.txt'):
            with open(file) as f:
                return([
                    float(x) for x in [
                        x.strip() for x in f.readlines()
                    ][1:]
                ])
            
    return(None)

In [ ]:
# args
outputs_root = os.path.expanduser("/data3/cnl/fmriprep/AllNewRun")
benchmark = {
    "software": "fmriprep",
    "run": 2
}
this_run = {
    "software": "C-PAC",
    "version": "1.6.2",
    "config": "on_nuisance_new"
}
feature_headers = {
    'GS': {
        'C-PAC': 'GlobalSignalMean0',
        'fmriprep': 'global_signal'
    },
    'CSF': {
        'C-PAC': 'CerebrospinalFluidMean0',
        'fmriprep': 'csf'
    },
    'WM': {
        'C-PAC': 'WhiteMatterMean0',
        'fmriprep': 'white_matter'
    },
    'CompCor': {
        'C-PAC': 'CompCorPC',
        'fmriprep': 'CompCor_comp_cor_0'
    }
}

In [ ]:
# Load config
with open('../configs/benchmarks.yml', 'r') as config_file:
    benchmarks = yaml.safe_load(config_file)

with open('../configs/cpac.yml', 'r') as config_file:
    config_settings = yaml.safe_load(config_file)
    
cpac_dir = os.path.abspath(os.path.join(os.getcwd(), *[os.pardir for i in range(3)]))    
n_cpus = 10
pipeline_config_path = os.path.join(cpac_dir, 'dev/docker_data/')
pipeline_file = 'default_pipeline.yml'
pipeline_name = pipeline_file.split('.yml')[0]
tag = git.Repo(search_parent_directories=True).head.object.hexsha[:7]
outputs_dir = os.path.join(config_settings['this_run_outputs'], tag)
benchmark_path = benchmarks[benchmark['software']][benchmark['run']]
if 'this_run' not in vars():
    this_run = {
        "software": "C-PAC",
        "version": tag,
        "config": pipeline_name
    }
sub_list = generate_subject_list_for_range(
    (
        config_settings['subjects']['start'],
        config_settings['subjects']['stop']
    ),
    (
        config_settings['sessions']['start'],
        config_settings['sessions']['stop']
    )
)
var_list = regressor_list + motion_list
if this_run['software'] not in benchmarks:
    benchmarks[this_run['software']] = {}
if this_run['version'] not in benchmarks[this_run['software']]:
    benchmarks[this_run['software']][this_run['version']] = {}
if this_run['config'] not in benchmarks[this_run['software']][this_run['version']]:
    benchmarks[this_run['software']][this_run['version']][this_run['config']] = "/".join([
        tag,
        pipeline_name
    ])
this_run_path = benchmarks[this_run['software']][this_run['version']][this_run['config']]
# with open('../configs/benchmarks.yml', 'w') as config_file:
#     config_file.write(yaml.dump(benchmarks))
corrs = np.zeros((len(sub_list), len(var_list)))

In [ ]:
# Load (new) C-PAC data & (versus) fmriprep data
cpac_paths = get_paths(
    feature="nuisance",
    sub_list=sub_list,
    run_path=os.path.join(outputs_root, this_run_path),
    software="C-PAC"
)
fmriprep_paths = get_paths(
    feature="nuisance",
    sub_list=sub_list,
    run_path=os.path.join(outputs_root, benchmark_path),
    software="fmriprep"
)

# Compute correlations
for var in regressor_list:
    print(var)
    cpac_file = cpac_paths['regressors']
    fmriprep_file = fmriprep_paths['regressors']
    cpac_data = read_feature(cpac_file, var, "C-PAC")
    fmriprep_data = read_feature(fmriprep_file, var, "fmriprep")
    

for var in motion_list:
    print(var)
    cpac_file = cpac_paths['fd']
    fmriprep_file = fmriprep_paths['fd']
    cpac_data = read_feature(cpac_file, var, "C-PAC")
    fmriprep_data = read_feature(fmriprep_file, var, "fmriprep")

In [ ]:
cpac_paths

---
* :snake: afnipython: `git+https://github.com/leej3/template_making.git`

---

In [ ]:
if isinstance(cpac_data, np.ndarray) and cpac_data.shape == fmriprep_data.shape:
    corr, _ = pearsonr(cpac_data.flatten(), fmriprep_data.flatten())
elif len(cpac_data) == len(fmriprep_data):
    corr, _ = pearsonr(cpac_data, fmriprep_data)
else:
    corrs[num_sub][num_var] = float('nan')

print('Running subject: ' + sub + ' ' + var + ' correlation score: ' + str(corr))
corrs[num_sub][num_var] = round(corr, 3)

In [ ]:
# Save matrix
sio.savemat('corrs.mat', {'corrs':corrs})

---

In [ ]:
# pipeline_comp_regressors.py

import os, sys, getopt, glob
import numpy as np
import pandas as pd 
import nibabel as nb
from afni_python import lib_afni1D as LAD
import scipy.io as sio
from scipy.stats import pearsonr
from nipype.interfaces import afni, fsl

def main(argv):
    options = "hc:f:"
    long_options = ["help", "cpac_path", "fmriprep_path"]

    try:
        opts, _ = getopt.getopt(argv, options, long_options)
    except getopt.error as err:
        print(str(err))
        print('pipeline_comp_regressor.py -c <cpac path> -f <fmriprep path>')
        sys.exit(2) 

    for opt, arg in opts:
        if opt in ("-h", "--help"):
            print('-c <cpac path> -f <fmriprep path>')
            sys.exit() 
        elif opt in ("-c", "--cpac_path"):
            cpac_path = arg
        elif opt in ("-f", "--fmriprep_path"): 
            fmriprep_path = arg  

    sub_list = range(25427,25457)
    ses_list = ['a']
    regressor_list = ['CSF', 'WM', 'GS', 'tCompCor0', 'aCompCor0', 'aCompCor1', 'aCompCor2', 'aCompCor3', 'aCompCor4']
    movement_list = ['FD']
    var_list = regressor_list + movement_list

    corrs = np.zeros((len(sub_list), len(var_list)))

    for num_sub, sub in enumerate(sub_list):

        sub = '00'+str(sub)

        for num_ses, ses in enumerate(ses_list):
            print(cpac_path + '/working/resting_preproc_sub-' + sub + ses + '_ses-1/nuisance_0_0/_*/*/build*/*1D')
            try:
                cpac_path_list = [glob.glob(cpac_path + '/working/resting_preproc_sub-' + sub + ses + '_ses-1/nuisance_0_0/_*/*/build*/*1D')[0]] * 9 + [ # CSF WM GS regressors
                    glob.glob(cpac_path + '/output/*/sub-' + sub + ses + '_ses-1/frame_wise_displacement_power/*/*')[0]] # frame wise displacement power 

                fmriprep_path_list = [fmriprep_path + '/output/fmriprep/sub-' + sub + ses + '/func/sub-' + sub + ses + '_task-rest_run-1_desc-confounds_regressors.tsv'] * 9 + [ # regressor
                    fmriprep_path + '/working/fmriprep_wf/single_subject_' + sub + ses + '_wf/func_preproc_task_rest_run_1_wf/bold_confounds_wf/fdisp/fd_power_2012.txt'] 

                print(len(cpac_path_list))
                display(cpac_path_list)
                print(len(fmriprep_path_list))
                display(fmriprep_path_list)
            except:
                print("👻⃠")

main([
    '-c', os.path.join(outputs_root, this_run_path),
    '-f', os.path.join(outputs_root, benchmark_path)
])

In [ ]:
        for num_var, var in enumerate(var_list):
            print str(num_var) + ' ' + var
            cpac_file = cpac_path_list[num_var]
            fmriprep_file = fmriprep_path_list[num_var]

            print cpac_file
            print fmriprep_file
            # read C-PAC files
            if var in regressor_list:
                data = LAD.Afni1D(cpac_file)
                header = data.header[-1]
                header_list = header.split('\t')
                if var == 'CSF':
                    cpac_data = data.mat[header_list.index('CerebrospinalFluidMean0')]
                elif var == 'WM':
                    cpac_data = data.mat[header_list.index('WhiteMatterMean0')]
                elif var == 'GS':
                    cpac_data = data.mat[header_list.index('GlobalSignalMean0')] # find string start with GS
                elif 'aCompCor' in var:
                    cpac_data = data.mat[header_list.index(var[:-1]+'PC'+var[-1])]  
                elif 'tCompCor' in var:
                    cpac_data = data.mat[header_list.index(var[:-1]+'PC'+var[-1])]                        
            elif var in movement_list:
                c = LAD.Afni1D(cpac_file)
                cpac_data = c.mat
                cpac_data = cpac_data[0][1:] # cpac and fmriprep dims are different

            # read fmriprep files
            if '.tsv' in fmriprep_file:
                data = pd.read_csv(fmriprep_file, sep='\t')
                if var == 'CSF':
                    fmriprep_data = data['csf']
                elif var == 'WM':
                    fmriprep_data = data['white_matter']
                elif var == 'GS':
                    fmriprep_data = data['global_signal']
                elif 'CompCor' in var:
                    fmriprep_data = data[var[0]+'_comp_cor_0'+var[-1]]
            elif '.txt' in fmriprep_file:
                with open(fmriprep_file) as f:
                    fmriprep_data = f.readlines()
                fmriprep_data = [x.strip() for x in fmriprep_data]
                fmriprep_data = fmriprep_data[1:]
                fmriprep_data = [float(x) for x in fmriprep_data]

            if isinstance(cpac_data, np.ndarray) and cpac_data.shape == fmriprep_data.shape:
                corr, _ = pearsonr(cpac_data.flatten(), fmriprep_data.flatten())
            elif len(cpac_data) == len(fmriprep_data):
                corr, _ = pearsonr(cpac_data, fmriprep_data)
            else:
                corrs[num_sub][num_var] = float('nan')

            print 'Running subject: ' + sub + ' ' + var + ' correlation score: ' + str(corr)
            corrs[num_sub][num_var] = round(corr, 3)

sio.savemat('corrs.mat', {'corrs':corrs})

if __name__ == "__main__":
main(sys.argv[1:])    